In [ ]:
import csv
import matplotlib.pyplot as plt
import numpy
import random
import math
from scipy.spatial import Voronoi, voronoi_plot_2d

'''
------------------------------------------------------------------------------------------------------------
    STARTER CODE 
------------------------------------------------------------------------------------------------------------
'''

given_points = []
with open('dataset.csv', 'r') as input_file:
    csv_reader = csv.reader(input_file)
    for row in csv_reader:
        x = float(row[0])
        y = float(row[1])
        given_points.append([x, y])

given_points = numpy.array(given_points)


In [ ]:
'''
------------------------------------------------------------------------------------------------------------
    DEFINING SOME USEFUL FUNCTIONS
------------------------------------------------------------------------------------------------------------
'''
# Function to calculate square of distance between 2 points
def distance(point1, point2):
    return (point1[0]-point2[0])**2 + (point1[1]-point2[1])**2


# Function to select K-means randomly
def get_k_means(K):
    indices_list = []
    for i in range(1000):
        indices_list.append(i)
    random.shuffle(indices_list)
    indices_in_clusters = []
    for i in range(K):
        indices_in_clusters.append([])
    for i in range(K):
        indices_in_clusters[i].append(indices_list[i])
    return indices_in_clusters


# Function to calculate mean of a cluster
def mean_of_cluster(points, indices):
    x = 0
    y = 0
    size = len(indices)
    for item in indices:
        x += points[item][0]
        y += points[item][1]
    x /= size
    y /= size
    return ([x, y])


# Function to calculate mean's of all clusters
def get_mean_all_clusters(points, point_indices_in_clusters):
    means_of_clusters = []
    for row in point_indices_in_clusters:
        mean = mean_of_cluster(points, row)
        means_of_clusters.append(mean)
    return means_of_clusters


# Function to calculate new cluster assigned to point
def get_current_cluster_of_point(index, point_indices_in_clusters):
    count = 0
    for row in point_indices_in_clusters:
        if index in row:
            return count
        count += 1
    return 0 # if not present in any cluster just returning 0


# k-means or lloyd's algorithm
def k_means_algorithm(points, K):
    point_indices_in_clusters = get_k_means(K)
    means_of_clusters = get_mean_all_clusters(points, point_indices_in_clusters)
    errors = []
    
    while(1):
        new_point_indices_in_clusters = []
        for i in range(K):
            new_point_indices_in_clusters.append([])
        flag = False
        total_error_in_current_iteration = 0
        for point_index in range(1000):
            current_cluster = get_current_cluster_of_point(point_index, point_indices_in_clusters)
            min_distance_from_cluster = distance(points[point_index], means_of_clusters[current_cluster])
            for cluster_index in range(K):
                distance_from_cluster = distance(points[point_index], means_of_clusters[cluster_index])
                if distance_from_cluster < min_distance_from_cluster:
                    flag = True
                    min_distance_from_cluster = distance_from_cluster
                    current_cluster = cluster_index

            new_point_indices_in_clusters[current_cluster].append(point_index)
            total_error_in_current_iteration += min_distance_from_cluster
        
        errors.append(total_error_in_current_iteration)
        
        #now all elements have been assigned to new clusters
        means_of_clusters = get_mean_all_clusters(points, new_point_indices_in_clusters)
        point_indices_in_clusters = new_point_indices_in_clusters.copy()

        #check for convergence
        if flag == False:
            break
    return (point_indices_in_clusters, means_of_clusters, errors)


# Function to plot clusters and the error funciton
def plot_clusters_and_error_function(points, point_indices_in_clusters, means_of_clusters, errors, iteration):
    colors = ['r', 'g', 'b', 'c', 'm', 'y', 'k']
    plt.figure(figsize = (13, 5))
    plt.subplot(1, 2, 1)
    x = []
    y = []
    for i, cluster in enumerate(point_indices_in_clusters):
        x = [points[index][0] for index in cluster]
        y = [points[index][1] for index in cluster]
        plt.scatter(x, y, c=colors[i % len(colors)], label=f'Cluster {i}', marker='o')

    for i, mean in enumerate(means_of_clusters):
        plt.scatter(mean[0], mean[1],c = 'k', label= f'Mean of Cluster {i}', marker = '*', s = 200)

    plt.title('Clustered Data Points')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(range(0, len(errors)), errors, label=f'Error function', marker = 'o', linestyle = '-')
    plt.title('Error Function w.r.t iterations')
    plt.xlabel('Iterations')
    plt.ylabel('Error')
    plt.legend()

    plt.tight_layout()
    name = f"Clusters_Error_Plots_Randomization{iteration}.png"
    plt.savefig(name)

    # plt.show()

In [ ]:
'''
----------------------------------------------------------------------------------------------------------
    PART (i)
----------------------------------------------------------------------------------------------------------
'''
for i in range(5):
    cluster_point_indices, means_of_clusters, errors = k_means_algorithm(given_points, 2)
    plot_clusters_and_error_function(given_points, cluster_point_indices, means_of_clusters, errors, i)

print("Saved plots in the name format 'Clusters_Error_Plots_Randomization(num)}.png' for part i\n")


In [ ]:
'''
---------------------------------------------------------------------------------------------------------
    DEFINING SOME USEFUL FUNCTIONS 
---------------------------------------------------------------------------------------------------------
'''
# Function to get k-means using seed to fix the randomization of indices(points)
def get_k_means_using_seed(K):
    indices_list = []
    for i in range(1000):
        indices_list.append(i)
    random.seed(180) #for fixing the random initialization
    random.shuffle(indices_list)
    indices_in_clusters = []
    for i in range(K):
        indices_in_clusters.append([])

    for i in range(K):
        indices_in_clusters[i].append(indices_list[i])
    return indices_in_clusters


# k-means or lloyd's algorithm by taking fixed randomization(for taking k-means) using seed()
def k_means_algorithm_using_seeded_initialization(points, K):
    point_indices_in_clusters = get_k_means_using_seed(K)
    means_of_clusters = get_mean_all_clusters(points, point_indices_in_clusters)

    while(1):
        new_point_indices_in_clusters = []
        for i in range(K):
            new_point_indices_in_clusters.append([])
        flag = False
        for point_index in range(1000):
            current_cluster = get_current_cluster_of_point(point_index, point_indices_in_clusters)
            min_distance_from_cluster = distance(points[point_index], means_of_clusters[current_cluster])
            for cluster_index in range(K):
                distance_from_cluster = distance(points[point_index], means_of_clusters[cluster_index])
                if distance_from_cluster < min_distance_from_cluster:
                    flag = True
                    min_distance_from_cluster = distance_from_cluster
                    current_cluster = cluster_index

            new_point_indices_in_clusters[current_cluster].append(point_index)
        
        #now all elements have been assigned to new clusters
        means_of_clusters = get_mean_all_clusters(points, new_point_indices_in_clusters)
        point_indices_in_clusters = new_point_indices_in_clusters.copy()

        #check for convergence
        if flag == False:
            break

    return point_indices_in_clusters, means_of_clusters


# Function to calculate minimun x,y coordinates and maximum x,y coordinates of the given data
def get_data_range():
    x_max = given_points[0][0]
    x_min = given_points[0][0]
    y_max = given_points[0][1]
    y_min = given_points[0][1]
    for item in given_points:
        x_max = max(item[0],x_max)
        x_min = min(item[0], x_min)
        y_max = max(item[1], y_max)
        y_min = min(item[1], y_min)
    return (x_max, x_min, y_max, y_min)

# Function for plotting voronoi regions when K=2
def plot_voronoi_regions_K_is_2(points):
    x_max, x_min, y_max, y_min = get_data_range()
    point_indices_in_clusters, mean_of_clusters = k_means_algorithm_using_seeded_initialization(points, 2)

    colors = ['r', 'g', 'b', 'c', 'm', 'y', 'k']

    for i, cluster in enumerate(point_indices_in_clusters):
        x = [points[index][0] for index in cluster]
        y = [points[index][1] for index in cluster]
        plt.scatter(x, y, c=colors[i % len(colors)], label=f'Cluster {i}', marker='o', alpha = 0.6)
    for i, mean_of_cluster in enumerate(mean_of_clusters):
        x = mean_of_cluster[0]
        y = mean_of_cluster[1]
        plt.scatter(x, y, c = 'k', label = f'Mean {i}', marker = '*', s = 200)

    # Calculate the slope and midpoint of the line joining the two means
    slope = (mean_of_clusters[1][1] - mean_of_clusters[0][1])/(mean_of_clusters[1][0] - mean_of_clusters[0][0])
    mid_x = (mean_of_clusters[0][0] + mean_of_clusters[1][0])/2
    mid_y = (mean_of_clusters[0][0] + mean_of_clusters[1][0])/2
    perpendicular_slope = -1 / slope
    perpendicular_y_intercept = mid_y - perpendicular_slope * mid_x

    x_line = numpy.linspace(x_min, x_max, 100)
    y_line = perpendicular_slope * x_line + perpendicular_y_intercept
    plt.plot(x_line, y_line, '--', c='k', label='Separating Line')
    plt.title(f'Voronoi Regions for K = 2')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.xlim(x_min, x_max)
    plt.ylim(y_min, y_max) 
    name = f'voronoi_region_K_2.png'
    plt.savefig(name)
    # plt.show()


# Function to plot voronoi regions for a general case K(other than 2)
def plot_voronoi_regions(points, K):
    x_max, x_min, y_max, y_min = get_data_range()
    point_indices_in_clusters, mean_of_clusters = k_means_algorithm_using_seeded_initialization(points, K)
    
    voronoi_diagram = Voronoi(mean_of_clusters)
    voronoi_plot_2d(voronoi_diagram, show_vertices=False)
    colors = ['r', 'g', 'b', 'c', 'm', 'y', 'k']
    
    for i, cluster in enumerate(point_indices_in_clusters):
        x = [points[index][0] for index in cluster]
        y = [points[index][1] for index in cluster]
        plt.scatter(x, y, c=colors[i % len(colors)], label=f'Cluster {i}', marker='o', alpha = 0.6)
   
    for i, mean_of_cluster in enumerate(mean_of_clusters):
        x = mean_of_cluster[0]
        y = mean_of_cluster[1]
        plt.scatter(x, y, c = 'k', label = f'Mean {i}', marker = '*', s = 200)
    plt.title(f'Voronoi Regions for K = {K}')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.xlim(x_min, x_max)
    plt.ylim(y_min, y_max) 
    name = f'voronoi_region_K_{K}.png'
    plt.savefig(name)
    # plt.show()

In [ ]:
'''
-------------------------------------------------------------------------------------------------------
    PART (ii)
-------------------------------------------------------------------------------------------------------
'''
# Plotting voronoi regions for K = 2, 3, 4, 5
K_value_set = [2, 3, 4, 5]
for K in K_value_set:
    if K == 2:
        plot_voronoi_regions_K_is_2(given_points)
    else:
        plot_voronoi_regions(given_points, K)
print("Saved plots in name format 'voronoi_region_K_(value of K).png for part ii \n")



In [ ]:
'''
-------------------------------------------------------------------------------------------------------
    DEFINING SOME USEFUL FUNCTIONS
-------------------------------------------------------------------------------------------------------
'''
# Function to get polynomial kernel matrix
def get_polynomial_kernel_matrix(d):
    kernel_matrix = numpy.zeros((1000, 1000))
    for i in range(1000):
        for j in range(1000):
            kernel_matrix[i][j] = (1 + numpy.dot(given_points[i],given_points[j].T)) ** d
    return kernel_matrix

# Function to get radial kernel matrix
def get_radial_kernel_matrix(sigma):
    kernel_matrix = numpy.zeros((1000, 1000))
    for i in range(1000):
        for j in range(1000):
            kernel_matrix[i][j] = math.exp(-1*(numpy.dot(((given_points[i]-given_points[j]).T),(given_points[i]-given_points[j])))/(2*(sigma**2)))
    return kernel_matrix


# Function to get new kernelized data points(after row normalization)
def get_new_kernelized_points(kernel_matrix, K):
    eigen_values, eigen_vectors = numpy.linalg.eigh(kernel_matrix)
    # Reversing eigen_values and eigen_vectors as eigen_values are in increasing order
    eigen_values = eigen_values[::-1]
    eigen_vectors = eigen_vectors[:, ::-1]

    #we need top K eigen_vectors for computing H
    H = []
    count = 0
    for row in eigen_vectors.T:
        H.append(row)
        count += 1
        if count == K:
            break
    H = numpy.array(H)
    H = H.T

    #Row normalizing
    row_normalized_H = []
    for row in H:
        denominator = 0
        normalized_row = []
        for item in row:
            denominator += item**2
        denominator = math.sqrt(denominator)
        for item in row:
            normalized_row.append(item/denominator)
        row_normalized_H.append(normalized_row)

    row_normalized_H = numpy.array(row_normalized_H)
    return row_normalized_H


# Function to plot clusters after spectral clustering
'''
    poly_or_radial = 1 for polynomial kernel
                     2 for radial kernel
'''
def plot_spectral_clusters(points, poly_or_radial, d_or_sigma, K):
    kernel_matrix = None
    if poly_or_radial == 1:
        kernel_matrix = get_polynomial_kernel_matrix(d_or_sigma)
    else:
        kernel_matrix = get_radial_kernel_matrix(d_or_sigma)

    new_kernelized_points = get_new_kernelized_points(kernel_matrix, K)
    point_indices_in_clusters, means_of_clusters, errors = k_means_algorithm(new_kernelized_points, K)

    colors = ['r', 'g', 'b', 'c', 'm', 'y', 'k']
    plt.figure()
    x = []
    y = []
    for i, cluster in enumerate(point_indices_in_clusters):
        x = [points[index][0] for index in cluster]
        y = [points[index][1] for index in cluster]
        plt.scatter(x, y, c=colors[i % len(colors)], label=f'Cluster {i}', marker='o')

    name = "Spectral_Clustering_"
    if poly_or_radial == 1:
        plt.title(f"Clustered Data Points using Polynomial Kernel (d = {d_or_sigma})")
        if d_or_sigma == 1e12:
            name += f'_polynomial_d_as_'
            name += '1e12'
        else:
            name += f'_polynomial_d_as_{d_or_sigma}'
    else:
        plt.title(f"Clustered Data Points using radial Kernel (sigma = {d_or_sigma})")
        if d_or_sigma == 1e12:
            name += f'_radial_d_as_'
            name += '1e12'
        else:
            name += f'_radial_d_as_{d_or_sigma}'
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.legend()
    name += '.png'
    plt.savefig(name)
    # plt.show()

In [ ]:
'''
---------------------------------------------------------------------------------------------------------
    PART - 3
---------------------------------------------------------------------------------------------------------
'''

'''
    poly_or_radial = 1 if polynomial kernel
                   = 2 if radial kernel
'''
plot_spectral_clusters(given_points, poly_or_radial = 1, d_or_sigma = 2, K = 2)
plot_spectral_clusters(given_points, poly_or_radial = 1, d_or_sigma = 3, K = 2)
plot_spectral_clusters(given_points, poly_or_radial = 1, d_or_sigma = 20, K = 2)
plot_spectral_clusters(given_points, poly_or_radial = 2, d_or_sigma = 2, K = 2)
plot_spectral_clusters(given_points, poly_or_radial = 2, d_or_sigma = 1e12, K = 2)
print("Saved plots for various d's(or sigma's) for polynomial(or radial) Kernels Respectively for part iii\n")


In [ ]:
'''
-------------------------------------------------------------------------------------------------------------
    DEFINING SOME USEFUL FUNCTIONS
-------------------------------------------------------------------------------------------------------------
'''
def custom_clustering(poly_or_radial, d_or_sigma, K):
    kernel_matrix = None
    if poly_or_radial == 1:
        kernel_matrix = get_polynomial_kernel_matrix(d_or_sigma)
    else:
        kernel_matrix = get_radial_kernel_matrix(d_or_sigma)

    eigen_values, eigen_vectors = numpy.linalg.eigh(kernel_matrix)
    eigen_values = eigen_values[::-1]
    eigen_vectors = eigen_vectors[:, ::-1]

    point_indices_in_clusters = []
    for i in range(K):
        point_indices_in_clusters.append([])

    for i in range(1000):
        current_cluster = 0
        current_max_v = (eigen_vectors.T)[0][i]
        for j, eigen_vector in enumerate(eigen_vectors.T):
            if j == K:
                break
            if eigen_vector[i] > current_max_v:
                current_max_v = eigen_vector[i]
                current_cluster = j
        point_indices_in_clusters[current_cluster].append(i)

    return point_indices_in_clusters


def plot_clusters_for_custom_method(points, poly_or_radial, d_or_sigma, K):
    point_indices_in_clusters = custom_clustering(poly_or_radial, d_or_sigma, K)

    colors = ['r', 'g', 'b', 'c', 'm', 'y', 'k']
    plt.figure()
    x = []
    y = []
    for i, cluster in enumerate(point_indices_in_clusters):
        x = [points[index][0] for index in cluster]
        y = [points[index][1] for index in cluster]
        plt.scatter(x, y, c=colors[i % len(colors)], label=f'Cluster {i}', marker='o')

    name = 'Custom_Clustering'
    
    if poly_or_radial == 1:
        plt.title(f"Clustered Data Points using Polynomial Kernel (d = {d_or_sigma})")
        if d_or_sigma == 1e12:
            name += f'_polynomial_d_as_'
            name += '1e12'
        else:
            name += f'_polynomial_d_as_{d_or_sigma}'
    else:
        plt.title(f"Clustered Data Points using radial Kernel (sigma = {d_or_sigma})")
        if d_or_sigma == 1e12:
            name += f'_radial_d_as_'
            name += '1e12'
        else:
            name += f'_radial_d_as_{d_or_sigma}'
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.legend()
    name += '.png'
    plt.savefig(name)
    # plt.show()



In [ ]:
'''
------------------------------------------------------------------------------------------------------------
    PART 4
------------------------------------------------------------------------------------------------------------
'''

plot_clusters_for_custom_method(given_points, poly_or_radial=1, d_or_sigma=2, K=2)
plot_clusters_for_custom_method(given_points, poly_or_radial=1, d_or_sigma=100, K=2)

plot_clusters_for_custom_method(given_points, poly_or_radial=2, d_or_sigma=2, K=2)
plot_clusters_for_custom_method(given_points, poly_or_radial=2, d_or_sigma=3, K=2)
plot_clusters_for_custom_method(given_points, poly_or_radial=2, d_or_sigma=100, K=2)
plot_clusters_for_custom_method(given_points, poly_or_radial=2, d_or_sigma=1e12, K=2)

print("Saved plots for various d's(or sigma's) for polynomial(or radial) Kernels Respectively for part iv\n")

